In [1]:
import warnings
warnings.filterwarnings("ignore")

import newspaper
from newspaper.google_news import GoogleNewsSource

import pandas as pd

04/18/2024 05:28:58 PM - Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
04/18/2024 05:28:58 PM - NumExpr defaulting to 8 threads.


In [2]:
articles_df = pd.DataFrame(columns = ['link', 'title', 'text'])

source = GoogleNewsSource(
    number_threads = 8)

source.build(top_news=False, keyword = 'Data Science')

for a in source.articles:
    try:
        a.download()
        a.parse()
        dict = pd.DataFrame({'link':[a.url],
                             'title':[a.title],
                             'text':[a.text]})
        
        articles_df = pd.concat([articles_df, dict], ignore_index=True)
    except:
        pass
    
articles_df

04/18/2024 05:29:55 PM - get_html_status(): bad status code 403 on URL: https://www.wsj.com/articles/gvs-newest-life-sciences-general-partner-pursues-precision-treatments-0a672296, html: 
  <!doctype html><!--GRAND CANYON PREBID -->
  <html lang=en>
    <head>
      <meta charSet='utf-8' />
      <meta name="description" content="We can’t find the page you are looking for."/>
    <met
04/18/2024 05:31:05 PM - get_html_status(): bad status code 403 on URL: https://www.fastcompany.com/91035960/data-science-most-innovative-companies-2024, html: <html><head><title>fastcompany.com</title><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS
04/18/2024 05:33:13 PM - error not enough image data while fetching: https://www.argusmedia.com/favicon.ico refer: https://www.argusmedia.com/en/solutions/analytics-and-forecasting/data-science-and-machine-learning


,link,title,text
0,https://towardsdatascience.com/the-4-hats-of-a...,The 4 Hats of a Full-Stack Data Scientist,This is the first article in a larger series o...
1,https://www.kdnuggets.com/5-free-courses-to-ma...,5 Free Courses to Master Math for Data Science,Image by storyset on Freepik\n\nWhen you’re le...
2,https://consumergoods.com/nestle-uses-ai-and-d...,Nestlé Uses AI and Data Science to Battle Envi...,Nestlé is brewing up data science- and artific...
3,https://fortune.com/education/articles/free-da...,10 free data analytics courses you can take on...,“It’s not hyperbole to say that data analytics...
4,https://towardsdatascience.com/simulated-data-...,"Simulated Data, Real Learnings : Simulating Sy...",INTRODUCTION\n\nSimulation is a powerful tool ...
...,...,...,...
90,https://towardsdatascience.com/how-to-make-you...,How to Make Yourself More Layoff-Proof as a Da...,Just when we thought that life is going back t...
91,https://www.vanderbilt.edu/undergrad-datascien...,Student Spotlight: Peyton Hudson ’26,Today we are highlighting Peyton Hudson (’26) ...
92,https://www.luc.edu/parkinson/about/newsevents...,Humanizing Data Science: Loyola University Chi...,Humanizing Data Science\n\nDaniel P. Smith\n\n...
93,https://www.kdnuggets.com/5-free-sql-courses-f...,5 Free SQL Courses for Data Science Beginners,Image by Author\n\nIf you’re interested in piv...


In [3]:
articles_df.to_csv('articles.csv', index = False)